In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [32]:
# read the training data
trainData = pd.read_csv('/kaggle/input/titanic/train.csv')

# read the verifying data
testData = pd.read_csv('/kaggle/input/titanic/test.csv')
# testData.head
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
# preprocessing the data
# fill the Null number of the age with the medium number of the age

trainData['Age'] = trainData['Age'].fillna(trainData['Age'].median())
# print(trainData.describe())

# transfer the Sex attributes into integer
sex_dict = {'male':0,'female':1}
trainData.replace({'Sex':sex_dict},inplace = True)
# trainData.head()

#transfer the Embarked attributes into integer
embark_dict = {'S':0,'C':1,'Q':2}
trainData.replace({'Embarked':embark_dict},inplace = True)
trainData.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,0.0
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,0.0


In [34]:
# construct the regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
# select the features that will be counted into the regression
# Since the passengerId, Name, Ticket and Fare may do little impact on the survival rate, we just decline these features
features = ['Pclass','Sex','Age','SibSp','Parch','Embarked','Survived']
trainData = trainData[features].dropna(how = 'any')
predictData = trainData[features[:-1]]
predictRes = trainData[features[-1]]
print(predictData)
predictData = predictData.dropna(how = 'any')
print(predictData)
model = LinearRegression()

kf = KFold(n_splits = 3)

result = []

for train,test in kf.split(predictData):
    X_train,X_test = predictData.iloc[train,:],predictData.iloc[test,:]
    y_train,y_test = predictRes.iloc[train],predictRes.iloc[test]
    
    model.fit(X_train,y_train)
    
    test_predict = model.predict(X_test)
    result.append(test_predict)
    
predictions = np.concatenate(result, axis = 0)
predictions[predictions>0.5] = 1
predictions[predictions<=0.5] = 0
# calculate the accuracy of the prediction
count = 0
res_list = list(predictRes)
print(len(res_list))
print(len(predictions))
for i in range(len(predictions)):
    if predictions[i] == res_list[i]: count += 1
print('Accuracy is %f' %(count / len(predictions)))
           



     Pclass  Sex   Age  SibSp  Parch  Embarked
0         3    0  22.0      1      0       0.0
1         1    1  38.0      1      0       1.0
2         3    1  26.0      0      0       0.0
3         1    1  35.0      1      0       0.0
4         3    0  35.0      0      0       0.0
..      ...  ...   ...    ...    ...       ...
886       2    0  27.0      0      0       0.0
887       1    1  19.0      0      0       0.0
888       3    1  28.0      1      2       0.0
889       1    0  26.0      0      0       1.0
890       3    0  32.0      0      0       2.0

[889 rows x 6 columns]
     Pclass  Sex   Age  SibSp  Parch  Embarked
0         3    0  22.0      1      0       0.0
1         1    1  38.0      1      0       1.0
2         3    1  26.0      0      0       0.0
3         1    1  35.0      1      0       0.0
4         3    0  35.0      0      0       0.0
..      ...  ...   ...    ...    ...       ...
886       2    0  27.0      0      0       0.0
887       1    1  19.0      0      0

In [35]:
# make predictions to the testData
# preprocessing the data
# fill the Null number of the age with the medium number of the age

testData['Age'] = testData['Age'].fillna(testData['Age'].median())
# print(trainData.describe())

# transfer the Sex attributes into integer
sex_dict = {'male':0,'female':1}
testData.replace({'Sex':sex_dict},inplace = True)
# trainData.head()

#transfer the Embarked attributes into integer
embark_dict = {'S':0,'C':1,'Q':2}
testData.replace({'Embarked':embark_dict},inplace = True)
testData.head()

features = ['Pclass','Sex','Age','SibSp','Parch','Embarked']
verifyData = testData[features]
verifyData.head()

,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,0,34.5,0,0,2
1,3,1,47.0,1,0,0
2,2,0,62.0,0,0,2
3,3,0,27.0,0,0,0
4,3,1,22.0,1,1,0


In [76]:
res = model.predict(verifyData)
res[res>0.5] = 1
res[res<=0.5] = 0

output = pd.DataFrame({'PassengerId':testData.PassengerId,'Survived':res})
# # output.head()
# print(output.dtypes)
output['Survived'] = output['Survived'].astype(int)


output.to_csv('submission.csv',index = False)

print(output)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
